First, we'll import the Bioverse code:

In [ ]:
# Append the main "Bioverse" directory to the system path
import sys
sys.path = ['../'] + sys.path

# Import the "Bioverse" modules
from bio import analysis, classes, plots

# Import pyplot (for making plots later) and adjust some of its settings
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 20.

Next, we'll simulate a sample of planets within 30 parsecs using the Chabrier (2003) IMF to create the stars and a modification of SAG13 planet occurrence rates to create the planetary systems.

In [ ]:
# Load the "Generator" object
generator = classes.Generator('IMF_SAG13')

# Run the generator to create a simulated planet sample within 30 parsecs
sample = generator.generate(d_max=30)

We can use the `pandas` Python package to display the simulated sample as a table (you can skip this if you don't have `pandas` installed).

In [ ]:
from pandas import DataFrame
DataFrame(sample)

This line will describe all of the parameters simulated by `Bioverse`.

In [ ]:
sample.legend()

Let's take a look at the statistics of planets in the sample. For example, the number of planets versus period and radius. Note that there is an upper limit on period and a lower limit on planet size (typically these planets are not detectable anyways).

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(16,8))

# Period histogram
P = sample['P']
bins = np.logspace(0, 4, 100)
ax[0].hist(P, bins=bins)
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].set_xlabel('Period (d)', fontsize=24)
ax[0].set_ylabel('Number of planets', fontsize=24)

# Radius histogram
R = sample['R']
bins = np.logspace(-0.5,1,100)
ax[1].hist(R, bins=bins)
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_xlabel('Radius ($R_\oplus$)', fontsize=24)
ax[1].set_ylabel('Number of planets', fontsize=24)

plt.subplots_adjust(wspace=0.3)

Now let's create a simulated data set for a LUVOIR-like direct imaging mission.

In [ ]:
# Load a new "Generator" object which draws targets from the Hipparcos stellar catalog instead of a stellar IMF
generator = classes.Generator('LUVOIR_SAG13')

# Load the "Survey" object which describes a LUVOIR-like survey (i.e. the appropriate
# coronagraphic field of view and limiting contrast ratio)
survey = classes.Survey('imaging')

# Simulate a new sample of planets
sample = generator.generate()

# Determine which simulated planets the survey can characterize. The new table `detected`
# contains only planets which lie within the field of view and are bright enough to detect.
detected = survey.compute_yield(sample)

# Finally, simulate a 10% measurement of the semi-major axis for each planet, and survey some of the
# planets to determine whether they have atmospheric H2O.
data = survey.observe(detected)

Note that the last three lines can be combined into one as follows:

In [ ]:
sample, detected, data = survey.quickrun(generator)

Let's take a look at the simulated data set by plotting which planets have H2O versus their insolation. You might notice that planets within the habitable zone (approx 0.3 < S < 1.1) are more likely to have water-rich atmospheres.

In [ ]:
# We have to compute the insolation first from the observed separation and stellar luminosity
data['S'] = data['L_st']/data['a']**2
x, y = data['S'], data['has_H2O']

# Now plot the water-rich/water-poor planets versus insolation (in log-space)
fig, ax = plt.subplots(figsize=(16,2))
ax.scatter(x,y)
ax.set_xscale('log')
ax.set_yticks([0,1])
ax.set_xlim([10,0.01])
ax.set_yticklabels(['no H$_2$O','has H$_2$O'],fontsize=24)
ax.set_xlabel('Insolation ($S_\oplus$)',fontsize=24)

# Highlight the boundaries of the habitable zone (water-rich planets should be more common here)
ax.axvline(1.12,linestyle='dotted',c='black',lw=3)
ax.axvline(0.36,linestyle='dotted',c='black',lw=3)

One hypothesis is that planets within the habitable zone are more likely to have atmospheric water vapor than those outside of it. Let's import this hypothesis and test it with the simulated data set.

Specifically, dynesty will calculate the Bayesian log-evidence (dlnZ) in favor of this "habitable zone hypothesis" over the null hypothesis that the presence of water vapor is uncorrelated with insolation. Values dlnZ > 3 are considered statistically significant.

In [ ]:
from hypothesis import h_HZ
results = h_HZ.fit(data)

print('Bayesian log-evidence in favor of the habitable zone hypothesis = {:.1f}'.format(results['dlnZ']))

The testability of this hypothesis primarily depends on what fraction of Earth-sized, habitable zone planets have water vapor (f_water_habitable). In the previous example, it is assumed to be ~80%. Let's test the hypothesis over a grid of values of f_water_habitable. For each value, we'll repeat the simulation 20 times to average over sampling error.

This may take a few minutes.

In [ ]:
f_water_habitable = np.logspace(-1, 0, 10)
results = analysis.test_hypothesis_grid(h_HZ, generator, survey, f_water_habitable=f_water_habitable, N=20, processes=8)

Run this cell to plot the typical log evidence versus f_water_habitable. Notice that as habitable planets become more common (from 10% to 100%), the evidence in favor of the habitable zone hypothesis becomes greater.

In [ ]:
plt.plot(f_water_habitable, results['dlnZ'].mean(axis=-1), lw=5)
plt.xlabel('Fraction of EECs with H2O', fontsize=20)
plt.ylabel('$\Delta$lnZ', fontsize=20)
plt.axhline(3, lw=5, c='black', linestyle='dashed')
plt.xscale('log')

A more useful metric than the average value of dlnZ is the survey's "statistical power", which is the likelihood that the survey could successfully test the hypothesis (dlnZ > 3). Run this cell to plot the statistical power versus the fraction of EECs with water vapor.

In [ ]:
power = analysis.compute_statistical_power(results, method='dlnZ', threshold=3)
plt.plot(f_water_habitable, power, lw=5)
plt.xlabel('Fraction of EECs with H2O', fontsize=20)
plt.ylabel('Statistical power', fontsize=20)
plt.axhline(0.8, lw=5, c='black', linestyle='dashed')
plt.xscale('log')